# Bavarian transmission grid aggregation

In this notebook, the bavarian transmission grid based on open street map data ([SciGrid](http://www.scigrid.de/pages/downloads.html)) is extracted and aggregated. The algorithm is based on the Bachelor's thesis "Automatic transmission grid aggregation for Bavaria ", supervised by Magdalena Dorfner.

## Needed data

The aggregation of the transmission system is applied on the [SciGRID Data project](http://www.scigrid.de/pages/downloads.html "SciGRID Downloads").

## Mathematical background

### Shunt susceptance

\begin{align*}
    b_{km}^{sh} = 2 \pi f c_{km}^{sh}
\end{align*}

### SIL
\begin{align*}
    SIL = \frac{V^2}{Z_c} = \frac{V^2}{\sqrt{\frac{X}{B_{sh}}}}
\end{align*}

## Code

### Package Import

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import re
import numpy as np
import math

geo = Nominatim()

pd.options.mode.chained_assignment = None  # default='warn'

### Settings

In [2]:
data_vertices = "Vertices_Scigrid.csvdata"
data_links = "Links_Scigrid.csvdata"
data_state = "Bayern"

### Vertices

In [3]:
vertices = pd.read_csv(data_vertices, index_col=0, usecols=[0,1,2,3,4,5,6,7])

# add new column address
vertices['address'] = np.nan
vertices['region'] = np.nan
vertices['state'] = np.nan

for lv in vertices.index:
    lat, lon = vertices.lat[lv], vertices.lon[lv]
    loc = geo.reverse('%s, %s' % (lat, lon), timeout=10)
    address = loc.address
    state = np.nan
    region = np.nan
    asplit = address.split(', ')
    try:
        # Problematic nodes with not standardized address string
        if (lv == 308) | (lv == 382)| (lv == 383):
            state = asplit[-4]
            region = asplit[-5]
        elif lv == 331:
            state = asplit[-2]
            region = asplit[-3]
        else:
            state = asplit[-3]
            region = asplit[-4]
    except:
        pass
    
    # Handling not standardized naming of administrative regionsb
    if region == 'OB':
        region = 'Oberbayern'
    elif region == 'NB':
        region = 'Niederbayern'
    elif region == 'OPf':
        region = 'Oberpfalz'
        
    vertices.loc[lv, 'address'] = address
    vertices.loc[lv, 'region'] = region
    vertices.loc[lv, 'state'] = state

### Links

In [4]:
links = pd.read_csv(data_links, index_col=[0,1,2])
indice = vertices.loc[vertices.state==data_state].index

# filter links which start and end in selected state 
state_links = links.loc[((links.index.get_level_values('v_id_1').isin(indice))) &
                   ((links.index.get_level_values('v_id_2').isin(indice)))]

index = pd.MultiIndex.from_tuples(state_links.index.values, names=['id', 'v1', 'v2'])
helpdf = pd.DataFrame(index=index, columns=['cap', 'reactance', 'shuntcap',
                                            'susceptance', 'capfactor', 'sil',
                                            'vin', 'vout', 'length'])
statelinks = pd.concat([helpdf, state_links.loc[:, 'voltage': 'frequency']], axis=1)


# add region to link dataframe
for (lv1, lv2, lv3) in index.values:
    ind = (lv1, lv2, lv3)
    statelinks.vin.loc[ind] = vertices.region[lv2]
    statelinks.vout.loc[ind] = vertices.region[lv3]

# extract transmission lines which are crossing a border
interlines = statelinks.loc[statelinks.vin != statelinks.vout]

### Calulation of physical values

In [5]:
# get values
interlines.reactance = state_links.loc[interlines.index].x_ohmkm
interlines.shuntcap = state_links.loc[interlines.index].c_nfkm
interlines.length = state_links.loc[interlines.index].length_m

# set mean values instead on nan
interlines.reactance.loc[interlines.reactance.isnull()] = interlines.loc[interlines.voltage==220000].reactance.mean()
interlines.shuntcap.loc[interlines.shuntcap.isnull()] = interlines.loc[interlines.voltage==220000].shuntcap.mean()

# calculcate shunt susceptance
interlines.susceptance = 2* math.pi* 50* interlines.shuntcap / (10**3)

# Filling the values for the loadability c
lengths = [0, 80000, 100000, 150000, 200000]
cfactors = [3, 2.75, 2.5, 2]

for lv in range(0, len(cfactors)):
    interlines.capfactor.loc[(interlines.length > lengths[lv]) &
                             (interlines.length <= lengths[lv+1])] = cfactors[lv]
    
# calculate surge impedance loading (SIL)
interlines.sil = (interlines.voltage / (10**3))**2 * (interlines.susceptance /
                                                      ((10**6) * interlines.reactance)) ** (1/2)

# calculates capacity
interlines.cap = interlines.sil * interlines.capfactor

# check for thermal limits
thermal_limits = {220000: 400, 380000: 1800}

interlines.cap.loc[(interlines.cap > interlines.voltage.map(thermal_limits) * interlines.cables / 3)
                   & (interlines.length < 80000)] = interlines.voltage.map(thermal_limits)

In [6]:
interlines

,,,cap,reactance,shuntcap,susceptance,capfactor,sil,vin,vout,length,voltage,cables,wires,frequency
id,v1,v2,,,,,,,,,,,,,
451,13,314,3594.87,0.125000,27.40,8.607964,3,1198.291295,Oberbayern,Schwaben,50053,380000,6.0,4.0,NaN
473,384,353,1497.86,0.250000,13.70,4.303982,2.5,599.145648,Mittelfranken,Oberbayern,100720,380000,6.0,2.0,50.0
478,332,331,387.309,0.384000,10.35,3.251548,2.75,140.839625,Niederbayern,Oberbayern,86154,220000,3.0,NaN,NaN
482,335,308,1797.44,0.250000,13.70,4.303982,3,599.145648,Oberfranken,Unterfranken,49572,380000,3.0,4.0,NaN
483,334,502,1797.44,0.250000,13.70,4.303982,3,599.145648,Oberfranken,Unterfranken,58294,380000,3.0,4.0,NaN
486,66,336,823.825,0.500000,6.85,2.151991,2.75,299.572824,Oberpfalz,Oberfranken,88422,380000,3.0,2.0,NaN
489,332,338,748.932,0.500000,6.85,2.151991,2.5,299.572824,Niederbayern,Oberpfalz,112852,380000,3.0,2.0,NaN
510,349,350,487.883,0.320000,11.50,3.612832,3,162.627591,Niederbayern,Mittelfranken,73783,220000,6.0,1.0,NaN
511,338,350,400,0.320000,11.50,3.612832,3,162.627591,Oberpfalz,Mittelfranken,62836,220000,3.0,2.0,NaN


### Deduplication and aggregation

In [7]:
def deduplicate_lines(df):
    lv = 0
    while lv < len(df)-1:
        # check for same in and output nodes
        if (df.iloc[lv,6] == df.iloc[lv+1,6]) & (df.iloc[lv,7] == df.iloc[lv+1,7]):
            # add capacities
            df.iloc[lv,0] = df.iloc[lv,0] + df.iloc[lv+1,0]
            df = df.drop(df.index[lv+1])
        else:
            lv +=1
            
    df_final = df
    return df_final

# exchange vin and vout in case of wrong order
helpbool = interlines.vin > interlines.vout
helpdf = interlines.vout.loc[helpbool]
interlines.vout.loc[helpbool] = interlines.vin.loc[helpbool]
interlines.vin.loc[helpbool] = helpdf

# sort transmission lines
intersorted = interlines.sort_values(['vin', 'vout'])

# aggregate!
deduplicate_lines(intersorted.reset_index().loc[:, 'cap':'vout'])

,cap,reactance,shuntcap,susceptance,capfactor,sil,vin,vout
0,487.883,0.320,11.50,3.612832,3,162.627591,Mittelfranken,Niederbayern
1,1497.86,0.250,13.70,4.303982,2.5,599.145648,Mittelfranken,Oberbayern
2,3295.3,0.250,13.70,4.303982,2.75,599.145648,Mittelfranken,Oberfranken
4,400,0.320,11.50,3.612832,3,162.627591,Mittelfranken,Oberpfalz
5,853.795,0.320,11.50,3.612832,2.75,162.627591,Mittelfranken,Unterfranken
7,6202.15,0.384,10.35,3.251548,2.75,140.839625,Niederbayern,Oberbayern
10,1101.03,0.500,6.85,2.151991,2.5,299.572824,Niederbayern,Oberpfalz
12,3594.87,0.125,27.40,8.607964,3,1198.291295,Oberbayern,Schwaben
13,1027.11,0.500,6.85,2.151991,2.75,299.572824,Oberfranken,Oberpfalz
15,3594.87,0.250,13.70,4.303982,3,599.145648,Oberfranken,Unterfranken
